# Ερώτημα Β: N-gram Language Models
Σε αυτή την άσκηση χρησιμοποιούμε ορισμένα κείμενα και πάλι από το Wall Street Journal. Τα κείμενα αυτά παρέχονται από τη βιβλιοθήκη nltk.corpus και στις προτάσεις τους έχει γίνει ήδη tokenization. Με τα κείμενα αυτά θα δημιουργήσουμε διάφορα μοντέλα διγραμμάτων ή τριγραμμάτων, θα κάνουμε k-smoothing και θα υπολογίσουμε το perplexity σε κάθε περίπτωση για να τα συγκρίνουμε μεταξύ τους.

#### Απαραίτητα Libraries
Ορισμένα από τα libraries που θα χρησιμοποιήσουμε είναι το NumPy το οποίο χρειάζεται για διάφορους αριθμητικούς υπολογισμούς και αρκετά libraries του nltk που χρησιμοποιούνται για τη δημιουργία των ngrams.

In [31]:
import numpy as np
from nltk.corpus import treebank
from nltk.lm.preprocessing import pad_both_ends
from nltk import ngrams

Η βιβλιοθήκη περιέχει 199 files. Σύμφωνα με την εκφώνηση τα πρώτα 170 files πρέπει να χρησιμοποιηθούν για το training των μοντέλων μας και τα υπόλοιπα 29 για το testing. Επομένως χωρίζουμε τα δεδομένα με αυτό τον τρόπο δημιουργώντας το training set και το test set.

In [32]:
files = treebank.fileids()

training_set_sentences = []
for file in files[:170]:
    for sentence in treebank.sents(file):
        training_set_sentences.append(sentence)

test_set_sentences = []
for file in files[170:]:
    for sentence in treebank.sents(file):
        test_set_sentences.append(sentence)

#### Δημιουργία του Vocabulary

Αρχικά ενώνω όλες τις προτάσεις του training set μεταξύ τους δημιουργώντας μία μεγάλη λίστα από tokens. Με τον τρόπο αυτό θα μπορέσω αργότερα να εξετάσω με ποια συχνότητα εμφανίζεται το κάθε token.

In [33]:
training_tokens = []
for sentence in training_set_sentences:
    training_tokens.extend(sentence)

Δημιουργώ ένα dictionary το οποίο έχει ως keys τα tokens και ως values τον αριθμό των εμφανίσεών τους στο training set.

In [34]:
tokens, counts = np.unique(training_tokens, return_counts=True) 
tokens_dict = dict(zip(tokens, counts))

Στη συνέχεια, σύμφωνα με την εκφώνηση, δημιουργώ το Vocabulary αφαιρώντας όλα τα tokens που εμφανίζονται κάτω από 3 φορές.

In [35]:
L = []
for key, value in dict(tokens_dict).items():
    if value >= 3:
        L.append(key)

print(L)        

['!', '#', '$', '%', '&', "'", "''", "'d", "'ll", "'m", "'re", "'s", "'ve", '*', '*-1', '*-2', '*-25', '*-3', '*-4', '*-5', '*-52', '*-6', '*-64', '*-7', '*-73', '*-80', '*?*', '*EXP*-1', '*EXP*-2', '*EXP*-3', '*ICH*-1', '*ICH*-2', '*ICH*-3', '*ICH*-4', '*PPA*-3', '*RNR*-1', '*RNR*-2', '*T*-1', '*T*-2', '*T*-3', '*T*-4', '*T*-5', '*U*', ',', '-', '--', '-LCB-', '-LRB-', '-RCB-', '-RRB-', '.', '...', '0', '0.1', '0.25', '1', '1,000', '1,500', '1.1', '1.2', '1.5', '1.65', '10', '10,000', '10-day', '10-year', '10.2', '100', '100,000', '11', '12', '12-point', '12.5', '120', '125', '13', '130', '14', '140', '15', '15,000', '150', '16', '16.7', '17', '17.3', '18', '180', '19', '1929', '195', '1970', '1972', '1976', '1977', '1979', '1980s', '1981', '1982', '1983', '1984', '1985', '1986', '1987', '1988', '1989', '1989-90', '1990', '1991', '1992', '1993', '1994', '1999', '1\\/2', '1\\/4', '1\\/8', '2', '2,000', '2,500', '2.1', '2.2', '2.50', '2.7', '20', '20,000', '200', '2009', '2019', '2029',

Εδώ ελέγχω πόσα tokens αφαιρέθηκαν συνολικά από το vocabulary γιατί είχαν εμφανιστεί λιγότερες από 3 φορές.

In [36]:
print('Out of', len(tokens_dict), 'total tokens', len(tokens_dict)-len(L), 'tokens were removed from the training dictionary becuase they appeared less than 3 times in our corpus.')

Out of 11683 total tokens 8010 tokens were removed from the training dictionary becuase they appeared less than 3 times in our corpus.


Στη συνέχεια, σύμφωνα με τις οδηγίες τις εκφώνησης, αντικαθιστούμε όλα τα tokens που τελικά δεν μπήκαν στο τελικό μας vocabulary με το token **\<UNK>**. Αυτό θα γίνει τόσο στο training όσο και στο test set. Για τη διευκόλυνση της διαδικασίας αυτής χρησιμοποιούμε ένα function που αντικαθιστά όλα τα tokens που πρέπει να αντικατασταθούν την οποία καλούμε τόσο για το training όσο και για το test set.

In [37]:
def replaceLowFrequencyTokensWithUNK(set, vocabulary):
    new_set = []
    for sentence in set:
        new_sentence = []
        for token in sentence:
            if token not in vocabulary:
                new_token = '<UNK>'
            else:
                new_token = token
            new_sentence.append(new_token)
        new_set.append(new_sentence)
    return new_set

training_set_with_UNK = replaceLowFrequencyTokensWithUNK(training_set_sentences, L)
test_set_with_UNK = replaceLowFrequencyTokensWithUNK(test_set_sentences, L)


Ακολούθως προσθέτουμε τα tokens **\<BOS>** και **\<EOS>** στην αρχή και το τέλος της κάθε πρότασης, αντίστοιχα. Και πάλι αυτό θα πρέπει να γίνει και στο training και στο test set και γίνεται με τη βοήθεια της function που ακολουθεί. Στη function χρησιμοποιείται η pad_both_ends από τη βιβλιοθήκη nltk.lm.preprocessing, η οποία μας επιτρέπει να προσθέσουμε ένα token αρχής και ένα token τέλους κάθε πρότασης. 

In [38]:
def addBOSandEOS(set):
    padded_set = []
    for sentence in set:
        padded_sentence = list(pad_both_ends(sentence, n=2, pad_left=True, left_pad_symbol="<BOS>", pad_right=True, right_pad_symbol="<EOS>"))
        padded_set.append(padded_sentence)
    return padded_set

final_training_set = addBOSandEOS(training_set_with_UNK)
final_test_set = addBOSandEOS(test_set_with_UNK)

## Training και Smoothing
Το επόμενο κομμάτι είναι το training και το smoothing των ngram μοντέλων που θα δημιουργήσουμε. Για το σκοπό αυτό δημιουργώ μία συνάρτηση η οποία θα χρησιμοποιηθεί για τις διάφορες τιμές του k που ζητάει η εκφώνηση (0.01 και 1). Η συνάρτηση αυτή παίρνει ως είσοδο το training set, την τιμή του k (για το smoothing) και το λεξιλόγιο L και επιστρέφει τις πιθανότητες των bigrams και των unigrams μετά το smoothing. Στην πραγματικότητα το μοντέλο είναι οι πιθανότητες των bigrams στην έξοδο της συνάρτησης, ωστόσο χρειαζόμαστε και τις τιμές των unigrams για τον υπολογισμό του perplexity στο test set. Είναι σημαντικό να σημειωθεί επίσης ότι για το μέγεθος του vocabulary (V), πρέπει να προσθέσουμε +1 καθώς στις προτάσεις του training set αρκετές λέξεις έχουν αντικαστασταθεί από το ειδικό token \<UNK>.

In [39]:
def trainBigramModelWithKSmoothing(set, k, vocabulary):
    V = len(vocabulary)+1

    raw_unigrams = []
    for sentence in set:
        for token in sentence:
            raw_unigrams.append(token)

    unigrams, counts = np.unique(raw_unigrams, return_counts=True) 
    unigrams_dict = dict(zip(unigrams, counts))

    train_unigrams= {}
    for key,value in unigrams_dict.items():
        train_unigrams[key] = k/float(unigrams_dict[key]+k*V)

    bigrams_dict = {}
    for sentence in set:
        for i in range(len(sentence)):
            if i+1 == len(sentence): 
                break
            elif (sentence[i], sentence[i+1]) in bigrams_dict:
                bigrams_dict[(sentence[i], sentence[i+1])] += 1
            else:
                bigrams_dict[(sentence[i], sentence[i+1])] = 1

    train_bigrams= {}
    for key,value in bigrams_dict.items():
        train_bigrams[key] = (value+k)/float(unigrams_dict[key[0]]+k*V)
    
    return train_bigrams, train_unigrams

Καλώντας τη συνάρτηση που δημιουργήσαμε παραπάνω γίνεται το training και το smoothing των bigram models για τιμές του k 0.01 και 1.0, αντίστοιχα.

In [40]:
bigram_k1_model, unigrams_k1 = trainBigramModelWithKSmoothing(final_training_set, 1.0, L)
bigram_k001_model, unigrams_k001 = trainBigramModelWithKSmoothing(final_training_set, 0.01, L)

Στη συνέχεια δημιουργούμε μία νέα συνάρτηση για την εκπαίδευση των μοντέλων τριγραμμάτων. Και πάλι ως είσοδος δίνεται το training set, η τιμή του k και το vocabulary και αυτή τη φορά η συνάρτηση επιστρέφει το μοντέλο τριγραμμάτων (πιθανότητες) αλλά και τις πιθανότητες των διγραμμάτων οι οποίες θα χρησιμοποιηθούν αργότερα για τον υπολογισμό του perplexity. Και εδώ προστίθεται +1 στο μέγεθος του λεξιλογίου για το ειδικό token \<UNK>.

In [41]:
def trainTrigramModelWithKSmoothing(set, k, vocabulary):
    V = len(vocabulary)+1

    bigrams_dict = {}
    for sentence in set:
        for i in range(len(sentence)):
            if i+1 == len(sentence): 
                break
            elif (sentence[i], sentence[i+1]) in bigrams_dict:
                bigrams_dict[(sentence[i], sentence[i+1])] += 1
            else:
                bigrams_dict[(sentence[i], sentence[i+1])] = 1
    
    train_bigrams= {}
    for key,value in bigrams_dict.items():
        train_bigrams[key] = k/float(bigrams_dict[key]+k*V)

    trigrams_dict = {}
    for sentence in set:
        for i in range(len(sentence)):
            if i+2 == len(sentence): 
                break
            elif (sentence[i], sentence[i+1], sentence[i+2]) in trigrams_dict:
                trigrams_dict[(sentence[i], sentence[i+1], sentence[i+2])] += 1
            else:
                trigrams_dict[(sentence[i], sentence[i+1], sentence[i+2])] = 1   
    
    train_trigrams= {}
    for key,value in trigrams_dict.items():
        train_trigrams[key] = (value+k)/float(bigrams_dict[(key[0],key[1])]+k*V)
    
    return train_trigrams, train_bigrams

Καλώντας τη συνάρτηση που δημιουργήσαμε παραπάνω παίρνουμε τα μοντέλα τριγραμμάτων για τις 2 τιμές του k που ζητούνται από την εκφώνηση.

In [42]:
trigram_k1_model, bigrams_k1 = trainTrigramModelWithKSmoothing(final_training_set, 1.0, L)
trigram_k001_model, bigrams_k001 = trainTrigramModelWithKSmoothing(final_training_set, 0.01, L)

## Υπολογισμός του Perplexity και Evaluation των Μοντέλων
Στο επόμενο κομμάτι εκτιμάται η αποτελεσματικότητα κάθε μοντέλου στο test set που ξεχωρίσαμε προηγουμένως, δηλαδή στα τελευταία 29 κείμενα που κρατήσαμε από τα 199. Για τον υπολογισμό του perplexity χρησιμοποιείται ο τύπος που δίνεται στην εκφώνηση της άσκησης.

Αρχικά δημιουργούμε μία συνάρτηση για τον υπολογισμό του perplexity στα μοντέλα διγραμμάτων. Η συνάρτηση λαμβάνει ως είσοδο το test set, την τιμή του k, τις πιθανότητες των bigrams και τις πιθανότητες των unigrams από το μοντέλο μας, καθώς και το λεξιλόγιο και επιστρέφει το perplexity. Και εδώ, όπως και στις προηγούμενες δύο συναρτήσεις προστίθεται +1 στο μέγεθος του λεξιλογίου καθώς υπάρχει και το token \<UNK> το οποίο δεν είχαμε υπολογίσει δημιουργώντας το λεξιλόγιο αρχικά.

In [43]:
def evaluate_bigram_model(k, bigram_probs, unigram_probs, test_set, vocabulary):
    log_prob_sum = 0
    N = 0
    V = len(vocabulary) + 1
    for sentence in test_set:
        for i in range(1, len(sentence)):
            ngram = tuple(sentence[i-1:i+1])
            n_1_gram = ngram[:-1]
            if ngram in bigram_probs:
                prob = bigram_probs[ngram]
            elif n_1_gram in unigram_probs:
                prob = unigram_probs[n_1_gram]
            else:
                prob = k / (k * V)
            log_prob_sum += np.log(prob)
            N += 1
    perplexity = round(np.exp(-log_prob_sum / N), 3)
    
    return perplexity

Στη συνέχεια δημιουργείται και η συνάρτηση για το perplexity των μοντέλων για τα τριγράμματα. Η συνάρτηση αυτή παίρνει ως είσοδο τις πιθανότητες των trigrams και τις πιθανότητες των bigrams από το μοντέλο μας, την τιμή του k, το test set και το λεξιλόγιο και επιστρέφει την τιμή του perplexity.

In [44]:
def evaluate_trigram_model(k, trigram_probs, bigram_probs, test_set, vocabulary):
    log_prob_sum = 0
    N = 0
    V = len(vocabulary) + 1
    for sentence in test_set:
        for i in range(2, len(sentence)):
            ngram = tuple(sentence[i-2:i+1])
            n_1_gram = ngram[:-1]
            if ngram in trigram_probs:
                prob = trigram_probs[ngram]
            elif n_1_gram in bigram_probs:
                prob = bigram_probs[n_1_gram]
            else:
                prob = k / (k * V)
            log_prob_sum += np.log(prob)
            N += 1
    perplexity = round(np.exp(-log_prob_sum / N), 3)
    
    return perplexity

Χρησιμοποιώντας τη συνάρτηση για τα bigrams παίρνουμε τις τιμές του perplexity για τις 2 τιμές του k που εξετάστηκαν.

In [45]:
print('The perplexity of the bigram model with k=0.01 on the test data is:', evaluate_bigram_model(0.01, bigram_k001_model, unigrams_k001, final_test_set, L))
print('The perplexity of the bigram model with k=1.0 on the test data is:', evaluate_bigram_model(1, bigram_k1_model, unigrams_k1, final_test_set, L))

The perplexity of the bigram model with k=0.01 on the test data is: 98.011
The perplexity of the bigram model with k=1.0 on the test data is: 370.851


Αντίστοιχα παίρνουμε και τις τιμές για τα μοντέλα των trigrams.

In [46]:
print('The perplexity of the trigram model with k=0.01 on the test data is:', evaluate_trigram_model(0.01, trigram_k001_model, bigrams_k001, final_test_set, L))
print('The perplexity of the trigram model with k=1.0 on the test data is:', evaluate_trigram_model(1, trigram_k1_model, bigrams_k1, final_test_set, L))

The perplexity of the trigram model with k=0.01 on the test data is: 463.891
The perplexity of the trigram model with k=1.0 on the test data is: 1505.009


## Μετατροπή σε Πεζά Γράμματα και εκ Νέου Εκπαίδευση των Μοντέλων

Στο επόμενο κομμάτι της άσκησης θα πρέπει να μετατρέψουμε όλα τα κείμενα σε πεζά γράμματα και να εξετάσουμε τι συμβαίνει με το perplexity (αν αυξάνεται η μειώνεται). Αρχικά διαβάζουμε τις προτάσεις και μετατρέπουμε όλα τα tokens ένα προς 1 σε lower με χρήση του σχετικού function.

In [47]:
files = treebank.fileids()

training_set_sentences_lower = []
for file in files[:170]:
    for sentence in treebank.sents(file):
        new_sentence = []
        for word in sentence:
            new_word = word.lower()
            new_sentence.append(new_word)
        training_set_sentences_lower.append(new_sentence)

test_set_sentences_lower = []
for file in files[170:]:
    for sentence in treebank.sents(file):
        new_sentence = []
        for word in sentence:
            new_word = word.lower()
            new_sentence.append(new_word)
        test_set_sentences_lower.append(new_sentence)

Ακολούθως δημιουργούμε το λεξιλόγιο και παρατηρούμε ότι αυτή τη φορά είναι μικρότερο σε μέγεθος, καθώς αρκετά tokens της μορφής "Token" και "token" τώρα θα λογίζονται ως ένα token και επομένως ο αριθμός των διαφορετικών tokens θα μειωθεί.

In [48]:
training_tokens_lower = []
for sentence in training_set_sentences_lower:
    training_tokens_lower.extend(sentence)

tokens, counts = np.unique(training_tokens_lower, return_counts=True) 
tokens_dict_lower = dict(zip(tokens, counts))

L_lower = []
for key, value in dict(tokens_dict_lower).items():
    if value >= 3:
        L_lower.append(key)

print('Out of', len(tokens_dict_lower), 'total tokens', len(tokens_dict_lower)-len(L_lower), 'tokens were removed from the training dictionary becuase they appeared less than 3 times in our corpus.')

Out of 10730 total tokens 7258 tokens were removed from the training dictionary becuase they appeared less than 3 times in our corpus.


Στη συνέχεια αντικαθιστούμε και πάλι τα tokens που εμφανίζονται λιγότερο από 3 φορές με το ειδικό token \<UNK>.

In [49]:
training_set_with_UNK_lower = replaceLowFrequencyTokensWithUNK(training_set_sentences_lower, L_lower)
test_set_with_UNK_lower = replaceLowFrequencyTokensWithUNK(test_set_sentences_lower, L_lower)

Και ακολούθως προσθέτουμε τα \<BOS> και \<EOS> στην αρχή και το τέλος κάθε πρότασης.

In [50]:
final_training_set_lower = addBOSandEOS(training_set_with_UNK_lower)
final_test_set_lower = addBOSandEOS(test_set_with_UNK_lower)

Γίνεται η εκπαίδευση και το smoothing για τα μοντέλα διγραμμάτων με τιμές του k 0.01 και 1.0.

In [51]:
bigram_k1_model_lower, unigrams_k1_lower = trainBigramModelWithKSmoothing(final_training_set_lower, 1.0, L_lower)
bigram_k001_model_lower, unigrams_k001_lower = trainBigramModelWithKSmoothing(final_training_set_lower, 0.01, L_lower)

Και για τα μοντέλα τριγραμμάτων.

In [52]:
trigram_k1_model_lower, bigrams_k1_lower = trainTrigramModelWithKSmoothing(final_training_set_lower, 1.0, L_lower)
trigram_k001_model_lower, bigrams_k001_lower = trainTrigramModelWithKSmoothing(final_training_set_lower, 0.01, L_lower)

Και υπολογίζουμε το perplexity στα νέα test data για τα διγράμματα.

In [53]:
print('The perplexity of the bigram model with k=0.01 on the test data is:', evaluate_bigram_model(0.01, bigram_k001_model_lower, unigrams_k001_lower, final_test_set_lower, L_lower))
print('The perplexity of the bigram model with k=1.0 on the test data is:', evaluate_bigram_model(1, bigram_k1_model_lower, unigrams_k1_lower, final_test_set_lower, L_lower))

The perplexity of the bigram model with k=0.01 on the test data is: 100.63
The perplexity of the bigram model with k=1.0 on the test data is: 370.55


Και για τα τριγράμματα.

In [54]:
print('The perplexity of the trigram model with k=0.01 on the test data is:', evaluate_trigram_model(0.01, trigram_k001_model_lower, bigrams_k001_lower, final_test_set_lower, L_lower))
print('The perplexity of the trigram model with k=1.0 on the test data is:', evaluate_trigram_model(1, trigram_k1_model_lower, bigrams_k1_lower, final_test_set_lower, L_lower))

The perplexity of the trigram model with k=0.01 on the test data is: 461.861
The perplexity of the trigram model with k=1.0 on the test data is: 1470.946


#### Συμπεράσματα
Από την μετατροπή των κειμένων σε πεζά γράμματα παρατηρούμε ότι το perplexity και στα δύο μοντέλα διγραμμάτων αυξήθηκε, ενώ το perplexity στα μοντέλα τριγραμμάτων μειώθηκε. Λογικά, θα περιμέναμε ότι το perplexity του μοντέλου θα μειωνόταν καθώς πλέον διγράμματα όπως για παράδειγμα το ('Hello', 'world') και το ('hello', 'world') θα λογιζόνται πλέον ως το ίδιο δίγραμμα, γεγονός που θα επιτρέπει στο μοντέλο μας να καταλάβει ευκολότερα ότι μετά τη λέξη 'hello' θα πρέπει να μπει η λέξη 'world'.


Στα μοντέλα τριγραμμάτων που δημιουργήσαμε, βλέπουμε ότι το perplexity μειώθηκε όταν όλες οι λέξεις έγιναν πεζές, γεγονός που δείχνει ότι τα μοντέλα αυτά πράγματι βελτιώθηκαν. Στα μοντέλα διγραμμάτων, ωστόσο, το perplexity αυξήθηκε επομένως άλλοι παράγοντες επηρέασαν την απόδοση των μοντέλων αυτών. Για παράδειγμα, ορισμένες λέξεις μπορεί να χάνουν το νόημά τους όταν αλλάζει το capitalization, γεγονός που ενδεχομένως δημιουργεί σύγχυση (όπως ας πούμε 'Turkey' η χώρα, όταν δεν είναι κεφαλαίο το Τ μπορεί να εκλαμβάνεται ως 'turkey' γαλοπούλα). Αντίστοιχα προβλήματα μπορεί να προκύψουν και με ονόματα τα οποία εκλαμβάνονται ως λέξεις. Για παράδειγμα το όνομα 'Ray' όταν δεν είναι κεφαλαίο το R μπορεί να δημιουργεί σύγχυση με την ακτίνα 'ray'. Τέλος, σε κείμενα με πολλά σημεία στίξης μπορεί επίσης να παρατηρηθεί αύξηση του perplexity από τη μετατροπή των χαρακτήρων σε πεζούς.

## Δημιουργία Τυχαίων Προτάσεων από τα Μοντέλα

Στο τελευταίο κομμάτι της άσκησης θα πρέπει να δημιουργήσουμε προτάσεις από τα 4 μοντέλα που έχουμε φτιάξει παραπάνω. Για το λόγο αυτό δημιουργούμε δύο functions που θα χρησιμοποιήσουμε για την επίτευξη του παραπάνω στόχου: μία για διγράμματα και μία για τριγράμματα.

Στις συναρτήσεις δημιουργούμε ένα νεό dictionary από το οποίο αφαιρούνται όλα τα bigrams ή trigrams που περιέχουν το token \<UNK> το οποίο σύμφωνα με την εκφώνηση δεν θα πρέπει να εμφανίζεται στις προτάσεις μας. Στη συνέχεια ελέγχουμε αν η αρχική λέξη που ζητά ο χρήστης (start_word) υπάρχει σε κάποιο bigram ή trigram από το οποίο μπορεί να ξεκινήσει πρόταση. Εφόσον αυτό δεν ισχύει, η συνάρτηση τερματίζει και επιστρέφει μήνυμα λάθους.


Αν πράγματι υπάρχει πρόταση που μπορεί να ξεκινήσει με αυτή τη λέξη ξεκινά ένα while loop που ελέγχει κάθε φορά πιθανές επόμενες λέξεις και ακολούθως επιλέξει μία λέξη με βάση τα βάρη των bigrams ή trigrams ανάλογα με τη συνάρτηση. Η **if** που υπάρχει στο while loop έχει ως στόχο να αποφύγει τη δημιουργία infinite loop επιλέγοντας ένα bigram ή trigram που δεν μπορεί τελικά να οδηγήσει στο token \<EOS> για να τερματιστεί η πρόταση.


Τέλος, όταν το επόμενο bigram ή trigram που πρέπει να προστεθεί περιέχει το ειδικό token \<EOS> η πρόταση ολοκληρώνεται και επιστρέφεται στο χρήστη.

In [55]:
import random

def generate_bigram_sentence(bigrams_dict_input, start_word):
    
    bigrams_dict = {}
    for key, value in bigrams_dict_input.items():
        if key[0] != '<UNK>' and key[1] != '<UNK>':
            bigrams_dict[key] = value    
             
    can_generate_sentence = False
    for key in bigrams_dict.keys():
        if key[0] == '<BOS>' and key[1] == start_word:
            can_generate_sentence = True

    if can_generate_sentence == False:
        print('Bigram not found, impossible to start a sentence to with that word.')
        return

    sentence = ['<BOS>']
    sentence.append(start_word)
    current_word = start_word
    
    while 1:
        possible_next_words = []
        for word in bigrams_dict.keys():
            if word[0] == current_word:
                possible_next_words.append((word[1], bigrams_dict[word]))

        if len(possible_next_words) == 0:
            sentence = ['<BOS>']
            sentence.append(start_word)
            current_word = start_word
            continue

        next_word_options = [word[0] for word in possible_next_words]
        weights = [word[1] for word in possible_next_words]
        next_word = random.choices(next_word_options, weights=weights)[0]

        sentence.append(next_word)
        current_word = next_word

        if next_word == '<EOS>':
            break
    
    return ' '.join(sentence)


In [56]:
def generate_trigram_sentence(trigrams_dict_input, start_word):
    
    trigrams_dict = {}

    for key, value in trigrams_dict_input.items():
        if key[0] != '<UNK>' and key[1] != '<UNK>' and key[2] != '<UNK>':
            trigrams_dict[key] = value    
             
    can_generate_sentence = False
    for key in trigrams_dict.keys():
        if key[0] == '<BOS>' and key[1] == start_word:
            can_generate_sentence = True

    if can_generate_sentence == False:
        print('Trigram not found, impossible to start a sentence to with that word.')
        return

    sentence = ['<BOS>']
    sentence.append(start_word)
    
    current_word_1 = '<BOS>'
    current_word_2 = start_word
    
    while 1:
        possible_next_words = []

        for word in trigrams_dict.keys():
            if word[0] == current_word_1 and word[1] == current_word_2:
                possible_next_words.append((word[2], trigrams_dict[word]))

        if len(possible_next_words) == 0:
            sentence = ['<BOS>']
            sentence.append(start_word)
            current_word_1 = '<BOS>'
            current_word_2 = start_word
            continue

        next_word_options = [word[0] for word in possible_next_words]
        weights = [word[1] for word in possible_next_words]

        next_word = random.choices(next_word_options, weights=weights)[0]

        sentence.append(next_word)

        current_word_1 = current_word_2
        current_word_2 = next_word

        if next_word == '<EOS>':
            break
    
    return ' '.join(sentence)


##### 3 Τυχαίες Προτάσεις με το Μοντέλο Διγραμμάτων με k=0.01

In [57]:
for i in range(3):
    sentence = generate_bigram_sentence(bigram_k001_model, start_word='I')
    if sentence == None:
        break
    print(f"Sentence {i+1}: {sentence}")

Sentence 1: <BOS> I asked , Ohio , said 0 for them for his letter to call to Charles E. Trotter III and foreign assistance and 0 than $ 15,000 *U* plus the Justice Department and are no asbestos will be delivered *-2 only an issue was essentially buy back that *T*-1 Richard Nixon , and chief executive vice president of principal of persons are meant only after UAL stake . <EOS>
Sentence 2: <BOS> I 'd have a request and has been found *-2 to put out there 's $ 30 . <EOS>
Sentence 3: <BOS> I have wanted *-1 with ringers . * with a credit standing required * joining forces investors , and Mexico Fund Report , most respected floor of sugar , Calif. , Colo. , `` the announcer talks with constitutional authority dropped below a new stadium was one place * for which *T*-1 . <EOS>


##### 3 Τυχαίες Προτάσεις με το Μοντέλο Διγραμμάτων με k=1

In [58]:
for i in range(3):
    sentence = generate_bigram_sentence(bigram_k1_model, start_word='A')
    if sentence == None:
        break
    print(f"Sentence {i+1}: {sentence}")

Sentence 1: <BOS> A planned acquisition of all '' are real estate and 10-day suspension ; Romanee-Conti , particularly true . <EOS>
Sentence 2: <BOS> A lack of all issues , *-1 to an appropriations clause -LRB- priced *-1 much -- products like Contel Corp. 's New England , though probably not students and services that he has become the government payments with ringers 0 *T*-2 . <EOS>
Sentence 3: <BOS> A couple of the region 's program , it : An official . <EOS>


##### 3 Τυχαίες Προτάσεις με το Μοντέλο Τριγραμμάτων με k=0.01

In [60]:
for i in range(3):
    sentence = generate_trigram_sentence(trigram_k001_model, start_word='I')
    if sentence == None:
        break
    print(f"Sentence {i+1}: {sentence}")

Sentence 1: <BOS> I say `` * providing 0 the ban on virtually all of us -LRB- but now , however . <EOS>
Sentence 2: <BOS> I believe in the Chicago Mercantile Exchange , a further 25 % in 1991 to 7.458 % . <EOS>
Sentence 3: <BOS> I get the answers to these people . <EOS>


##### 3 Τυχαίες Προτάσεις με το Μοντέλο Τριγραμμάτων με k=1

In [61]:
for i in range(3):
    sentence = generate_trigram_sentence(trigram_k1_model, start_word='A')
    if sentence == None:
        break
    print(f"Sentence {i+1}: {sentence}")

Sentence 1: <BOS> A year ago , I 'd have a real bad day , may be low , 8 3\/4 % to $ 90 to $ 8.5 million *U* last year . <EOS>
Sentence 2: <BOS> A major concern about such practices , according to Ms. Poore , the lowest -- a product for upscale professionals . <EOS>
Sentence 3: <BOS> A bank spokeswoman also declined *-1 to be covered *-1 . <EOS>


## Τελικές Παρατηρήσεις

Όπως φαίνεται από τις τυχαίες προτάσεις που δημιουργούνται, γενικά τα μοντέλα τριγραμμάτων δημιουργούν πιο ανθρώπινες προτάσεις που βρίσκονται πιο κοντά στη φυσική γλώσσα. Ωστόσο, παρατηρούμε ότι αρκετές προτάσεις περιέχουν διάφορα σύμβολα και σημεία στίξης, γεγονός που όπως φαίνεται δημιουργεί θόρυβο και τελικά μειώνει την αποτελεσματικότητα του μοντέλου. Το γεγονός αυτό μπορεί να εξηγήσει ίσως γιατί το perplexity των μοντέλων διγραμμάτων μειώθηκε μετά την μετατροπή των χαρακτήρων σε πεζούς.